In [ ]:
train_file_path = 'sign_mnist_train2.csv'
test_file_path = 'test2.csv'

In [ ]:
train_file_path = 'https://raw.githubusercontent.com/Herbrax/Kaggle_MNIST_Sign/main/sign_mnist_train2.csv'
test_file_path = 'https://raw.githubusercontent.com/Herbrax/Kaggle_MNIST_Sign/main/test2.csv'

In [18]:
train_file_path = 'sign_mnist_train.csv'
test_file_path = 'test.csv'

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

# Setting a random seed for reproducibility
np.random.seed(42)
split_ratio = 0.8

###################### -- HELPER FUNCTIONS -- ######################

def separate_test_sets(file_path):
    test_data = pd.read_csv(file_path)
    test_a_columns = [col for col in test_data.columns if 'pixel_a' in col]
    test_b_columns = [col for col in test_data.columns if 'pixel_b' in col]
    test_a = test_data[test_a_columns]
    test_b = test_data[test_b_columns]
    test_a.columns = [col.replace('_a', '') for col in test_a.columns]
    test_b.columns = [col.replace('_b', '') for col in test_b.columns]
    return test_a, test_b

def predict_and_merge(model, test_a, test_b):
    preds_a = model.compute_predictions(test_a)
    preds_b = model.compute_predictions(test_b)
    merged_predictions = []
    for i in range(len(preds_a)):
        # Replacing 9 back to 24 if needed
        decoded_a = preds_a[i] if preds_a[i] != 9 else 24
        decoded_b = preds_b[i] if preds_b[i] != 9 else 24

        ascii_a = decoded_a + 65
        ascii_b = decoded_b + 65
        sum_pred = normalize_ascii_sum(ascii_a + ascii_b)
        merged_predictions.append((i, chr(sum_pred)))
    return merged_predictions


def predict_and_merge2(model, test_a, test_b):
    preds_a = model.compute_predictions(test_a)
    preds_b = model.compute_predictions(test_b)
    merged_predictions = []
    for i in range(len(preds_a)):
        decoded_a = preds_a[i] if preds_a[i] != 9 else 24
        decoded_b = preds_b[i] if preds_b[i] != 9 else 24
        sum_pred = decoded_a + decoded_b
        print(decoded_a,";",decoded_b,"----------------------",sum_pred)
        merged_predictions.append((i, sum_pred))
    return merged_predictions

def save_predictions_to_csv(filename, predictions):
    with open(filename, 'w') as file:
        file.write("id,label\n")
        for id, label in predictions:
            file.write(f"{id},{label}\n")

def normalize_ascii_sum(ascii_sum):
    while ascii_sum > 122:  # 'z' is ASCII 122
        ascii_sum -= 65  # 122 ('z') - 65 ('A') + 1
    return int(ascii_sum)

def plot_results_XGB(results):
    learning_rates = [x[0] for x in results]
    max_depths = [x[1] for x in results]
    accuracies = [x[2] for x in results]
    plt.figure(figsize=(10, 6))
    for lr in set(learning_rates):
        specific_lr_depths = [depth for depth, l_rate in zip(max_depths, learning_rates) if l_rate == lr]
        specific_acc = [acc for acc, l_rate in zip(accuracies, learning_rates) if l_rate == lr]
        plt.plot(specific_lr_depths, specific_acc, label=f'Learning Rate {lr}')
    plt.title('Accuracy for different max depths and learning rates')
    plt.xlabel('Max Depth')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()



###################### -- Data Handling -- ######################

mnist_sign_train = pd.read_csv(train_file_path)
mnist_sign_train['label'] = mnist_sign_train['label'].replace(24, 9)
mnist_sign_train = mnist_sign_train.sample(frac=1).reset_index(drop=True)
labels = mnist_sign_train['label'].values
features = mnist_sign_train.drop('label', axis=1).values

features_normalized = features / 255.0
split_index = int(split_ratio * len(features_normalized))
train_data, validation_data = features_normalized[:split_index], features_normalized[split_index:]
train_labels, validation_labels = labels[:split_index], labels[split_index:]

train_mean = train_data.mean(axis=0)
train_std = train_data.std(axis=0)
train_data_normalized = (train_data - train_mean) / train_std
validation_data_normalized = (validation_data - train_mean) / train_std

test_a, test_b = separate_test_sets(test_file_path)
normalized_test_a = (test_a.values / 255.0 - train_mean) / train_std
normalized_test_b = (test_b.values / 255.0 - train_mean) / train_std

###################### -- XGBOOST Implementation -- ######################

class XGBoostClassifier:
    def __init__(self, max_depth, eta, num_class):
        self.params = {
            'objective': 'multi:softmax',
            'num_class': num_class,
            'booster': 'gbtree',
            'eval_metric': 'merror',
            'eta': eta,
            'max_depth': max_depth,
        }

    def train(self, train_data, train_labels, validation_data, validation_labels):
        dtrain = xgb.DMatrix(train_data, label=train_labels)
        dval = xgb.DMatrix(validation_data, label=validation_labels)
        watchlist = [(dtrain, 'train'), (dval, 'validation')]
        self.model = xgb.train(self.params, dtrain, num_boost_round=200, evals=watchlist, early_stopping_rounds=20, verbose_eval=False)

    def compute_predictions(self, data):
        ddata = xgb.DMatrix(data)
        return self.model.predict(ddata)

    def compute_accuracy(self, preds, labels):
        return np.mean(preds == labels)

###################### -- TRAINING -- ######################

def startXGBOOST(train_data_normalized, train_labels, validation_data_normalized, validation_labels):
    learning_rates = [0.01, 0.05, 0.1, 0.2, 0.3]
    max_depths = [3, 4, 5, 6, 7, 8, 9, 10]
    results_XGB = []
    best_acc_xgb = 0
    best_lr = None
    best_depth = None
    num_class = len(np.unique(train_labels))

    for lr in learning_rates:
        for depth in max_depths:
            xgb_model = XGBoostClassifier(max_depth=depth, eta=lr, num_class=num_class)
            xgb_model.train(train_data_normalized, train_labels, validation_data_normalized, validation_labels)
            val_preds = xgb_model.compute_predictions(validation_data_normalized)
            acc = xgb_model.compute_accuracy(val_preds, validation_labels)
            print(f"LR: {lr} ; Depth: {depth} ; Accuracy: {acc * 100:.2f}%")
            if acc > best_acc_xgb:
                best_acc_xgb = acc
                best_lr = lr
                best_depth = depth
                print("New best")
            results_XGB.append((lr, depth, acc))

    #plot_results_XGB(results_XGB)

    best_xgb_model = XGBoostClassifier(max_depth=best_depth, eta=best_lr, num_class=num_class)
    best_xgb_model.train(train_data_normalized, train_labels, validation_data_normalized, validation_labels)

    return best_xgb_model,best_lr,best_depth

#best_xgb_model = startXGBOOST(train_data_normalized, train_labels, validation_data_normalized, validation_labels)

#final_predictions = predict_and_merge(best_xgb_model, normalized_test_a, normalized_test_b)

#save_merged_predictions_to_csv("merged_predictions.csv", final_predictions)


In [20]:
print(train_file_path)
print(test_file_path)
best_xgb_model, best_lr, best_depth = startXGBOOST(train_data_normalized, train_labels, validation_data_normalized, validation_labels)

sign_mnist_train.csv
test.csv
LR: 0.01 ; Depth: 3 ; Accuracy: 76.78%
New best
LR: 0.01 ; Depth: 4 ; Accuracy: 87.65%
New best
LR: 0.01 ; Depth: 5 ; Accuracy: 92.52%
New best
LR: 0.01 ; Depth: 6 ; Accuracy: 95.12%
New best


In [14]:
final_predictions = predict_and_merge(best_xgb_model, normalized_test_a, normalized_test_b)
save_predictions_to_csv(f"{best_lr}_{best_depth}.csv", final_predictions)

3.0 ; 6.0 ---------------------- 9.0
2.0 ; 22.0 ---------------------- 24.0
1.0 ; 5.0 ---------------------- 6.0
13.0 ; 16.0 ---------------------- 29.0
10.0 ; 17.0 ---------------------- 27.0
17.0 ; 23.0 ---------------------- 40.0
19.0 ; 12.0 ---------------------- 31.0
16.0 ; 0.0 ---------------------- 16.0
4.0 ; 15.0 ---------------------- 19.0
24 ; 0.0 ---------------------- 24.0
3.0 ; 3.0 ---------------------- 6.0
2.0 ; 19.0 ---------------------- 21.0
4.0 ; 7.0 ---------------------- 11.0
0.0 ; 0.0 ---------------------- 0.0
0.0 ; 0.0 ---------------------- 0.0
4.0 ; 19.0 ---------------------- 23.0
14.0 ; 24 ---------------------- 38.0
13.0 ; 4.0 ---------------------- 17.0
10.0 ; 8.0 ---------------------- 18.0
4.0 ; 16.0 ---------------------- 20.0
5.0 ; 15.0 ---------------------- 20.0
8.0 ; 8.0 ---------------------- 16.0
13.0 ; 4.0 ---------------------- 17.0
19.0 ; 22.0 ---------------------- 41.0
15.0 ; 22.0 ---------------------- 37.0
19.0 ; 3.0 ---------------------- 

In [ ]:
final_predictions2 = predict_and_merge2(best_xgb_model, normalized_test_a, normalized_test_b)
save_predictions_to_csv("numsum_preds.csv", final_predictions2)


#23;17     40                        ONLY WRONG ONE, 17 est prédit comme 18.
#18;23     41
#13;20     33
#12;11     23
#19;14     33